<a href="https://colab.research.google.com/github/rasoco/master_data_science/blob/master/Big%20Data/Clase%202/02_SparkSQL_DataFrames_workingme.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SparkSQL and DataFrames 

<a href = "http://yogen.io"><img src="http://yogen.io/assets/logo.svg" alt="yogen" style="width: 200px; float: right;"/></a>

## RDDs, DataSets, and DataFrames

RDDs are the original interface for Spark programming.

DataFrames were introduced in 1.3

Datasets were introduced in 1.6, and unified with DataFrames in 2.0

### Advantages of DataFrames:

from https://www.datacamp.com/community/tutorials/apache-spark-python:

> More specifically, the performance improvements are due to two things, which you’ll often come across when you’re reading up DataFrames: custom memory management (project Tungsten), which will make sure that your Spark jobs much faster given CPU constraints, and optimized execution plans (Catalyst optimizer), of which the logical plan of the DataFrame is a part.

## SparkSQL and DataFrames 


pyspark does not have the Dataset API, which is available only if you use Spark from a statically typed language: Scala or Java.

From https://spark.apache.org/docs/2.4.4/sql-programming-guide.html

> A DataFrame is a Dataset organized into named columns. It is conceptually equivalent to a table in a relational database or a data frame in R/Python, but with richer optimizations under the hood. DataFrames can be constructed from a wide array of sources such as: structured data files, tables in Hive, external databases, or existing RDDs. The DataFrame API is available in Scala, Java, Python, and R. In Scala and Java, a DataFrame is represented by a Dataset of Rows. In the Scala API, DataFrame is simply a type alias of Dataset[Row]. While, in Java API, users need to use Dataset&lt;Row> to represent a DataFrame.


### The pyspark.sql module

Important classes of Spark SQL and DataFrames:

* `pyspark.sql.SparkSession` Main entry point for DataFrame and SQL functionality.

* `pyspark.sql.DataFrame` A distributed collection of data grouped into named columns.

* `pyspark.sql.Column` A column expression in a DataFrame.

* `pyspark.sql.Row` A row of data in a DataFrame.

* `pyspark.sql.GroupedData` Aggregation methods, returned by DataFrame.groupBy().

* `pyspark.sql.DataFrameNaFunctions` Methods for handling missing data (null values).

* `pyspark.sql.DataFrameStatFunctions` Methods for statistics functionality.

* `pyspark.sql.functions` List of built-in functions available for DataFrame.

* `pyspark.sql.types` List of data types available.

* `pyspark.sql.Window` For working with window functions.

http://spark.apache.org/docs/2.4.4/api/python/pyspark.sql.html

https://spark.apache.org/docs/2.4.4/sql-programming-guide.html

## SparkSession

The traditional way to interact with Spark is the SparkContext. In the notebooks we get that from the pyspark driver.

From 2.0 we can use SparkSession to replace SparkConf, SparkContext and SQLContext

### If you are running this notebook in Google Colab

Copy the following to a code cell and run it. It will install and set up Spark for you.

```python
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://apache.uvigo.es/spark/spark-2.4.6/spark-2.4.6-bin-hadoop2.7.tgz
!tar -xf spark-2.4.6-bin-hadoop2.7.tgz
!pip install -q findspark

import os
import findspark
from pyspark.sql import SparkSession

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.6-bin-hadoop2.7"
findspark.init()
spark = SparkSession.builder.master("local[*]").getOrCreate()
```

In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://apache.uvigo.es/spark/spark-2.4.6/spark-2.4.6-bin-hadoop2.7.tgz
!tar -xf spark-2.4.6-bin-hadoop2.7.tgz
!pip install -q findspark pyspark==2.4.6
 
import os
import findspark
from pyspark.sql import SparkSession
 
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.6-bin-hadoop2.7"
findspark.init()
spark = SparkSession.builder.master("local[*]").getOrCreate()

!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://apache.uvigo.es/spark/spark-2.4.6/spark-2.4.6-bin-hadoop2.7.tgz
!tar -xf spark-2.4.6-bin-hadoop2.7.tgz
!pip install -q findspark pyspark==2.4.6


     |████████████████████████████████| 218.4MB 62kB/s 
     |████████████████████████████████| 204kB 47.3MB/s 


#### Passing other options to spark session:
    
    

In [3]:
spark

In [4]:
spark.sparkContext

<SparkContext master=local[*] appName=pyspark-shell>

We can check option values in the resulting session like this:

In [8]:
spark.sparkContext.getConf().getAll()

[('spark.driver.host', '21c206dd9641'),
 ('spark.rdd.compress', 'True'),
 ('spark.app.id', 'local-1593797212585'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.driver.port', '37007'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.app.name', 'pyspark-shell')]

### Creating DataFrames

SparkSession.createDataFrame: from an RDD, a list or a pandas.DataFrame.

In [12]:
import random
random.seed(42)
n = 20
races = random.choices(['elf', 'hobbit', 'orc'], k=n)
creatures = [(id_, race) for id_, race in zip(range(n), races)] #crear lista de tuplas
creatures

[(0, 'hobbit'),
 (1, 'elf'),
 (2, 'elf'),
 (3, 'elf'),
 (4, 'orc'),
 (5, 'orc'),
 (6, 'orc'),
 (7, 'elf'),
 (8, 'hobbit'),
 (9, 'elf'),
 (10, 'elf'),
 (11, 'hobbit'),
 (12, 'elf'),
 (13, 'elf'),
 (14, 'hobbit'),
 (15, 'hobbit'),
 (16, 'elf'),
 (17, 'hobbit'),
 (18, 'orc'),
 (19, 'elf')]

In [62]:
df = spark.createDataFrame(creatures)
df # columna con un nombre autogenerada _1 y _2 y un tipo

DataFrame[_1: bigint, _2: string]

In [14]:
df.take(5) #elementos son filas, tupla con nombres 

[Row(_1=0, _2='hobbit'),
 Row(_1=1, _2='elf'),
 Row(_1=2, _2='elf'),
 Row(_1=3, _2='elf'),
 Row(_1=4, _2='orc')]

In [15]:
from pyspark.sql import Row

Row(id_=4, race='elf')

Row(id_=4, race='elf')

In [73]:
df = spark.createDataFrame(creatures, schema=['id', 'race']) #dar nombre a las columnas
df

DataFrame[id: bigint, race: string]

In [74]:
df.show(5)

+---+------+
| id|  race|
+---+------+
|  0|hobbit|
|  1|   elf|
|  2|   elf|
|  3|   elf|
|  4|   orc|
+---+------+
only showing top 5 rows



In [18]:
df.printSchema() #campo que sea obligatorio (nullable)

root
 |-- id: long (nullable = true)
 |-- race: string (nullable = true)



### Creating DataFrames

* From RDDs
* from Hive tables
* From Spark sources: parquet (default), json, jdbc, orc, libsvm, csv, text
* Is inmutable


In [19]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


#### From RDDs

In [24]:
rdd = spark.sparkContext.textFile('/content/drive/My Drive/BigData Spark/Clase 1/coupon150720.csv.gz')

In [25]:
rdd.take(3)

['79062005698500,1,MAA,AUH,9W,9W,56.79,USD,1,H,H,0526,150904,OK,IAF0',
 '79062005698500,2,AUH,CDG,9W,9W,84.34,USD,1,H,H,6120,150905,OK,IAF0',
 '79062005924069,1,CJB,MAA,9W,9W,60.0,USD,1,H,H,2768,150721,OK,IAA0']

Si aplicamos map vamos a tener un rdd anidado saca, son agrupación en lista

si queremos romper la agrupación en línea para ello utilizaremos flatMap

In [27]:
split_lines = rdd.map(lambda line: line.split(',')).take(3)

In [28]:
spark.createDataFrame(split_lines)

DataFrame[_1: string, _2: string, _3: string, _4: string, _5: string, _6: string, _7: string, _8: string, _9: string, _10: string, _11: string, _12: string, _13: string, _14: string, _15: string]

### Inferring and specifying schemas

#### Fully specifying a schema

We need to create a `StructType` composed of `StructField`s. each of those specifies afiled with name, type and `nullable` properties. 

In [41]:
from pyspark.sql import types
types.BooleanType()

schema = types.StructType([types.StructField('id', types.IntegerType(), False), types.StructField('race', types.StringType())])
df = spark.createDataFrame(creatures, schema=schema)
df.printSchema() #no pueden ver valores nulos

root
 |-- id: integer (nullable = false)
 |-- race: string (nullable = true)



#### From csv files

We can either read them directly into dataframes or read them as RDDs and transform that into a DataFrame. This second way will be very useful if we have unstructured data like web server logs.

In [42]:
spark.read.csv('/content/drive/My Drive/BigData Spark/Clase 1/coupon150720.csv.gz', inferSchema=True)

DataFrame[_c0: bigint, _c1: int, _c2: string, _c3: string, _c4: string, _c5: string, _c6: double, _c7: string, _c8: int, _c9: string, _c10: string, _c11: string, _c12: int, _c13: string, _c14: string]

In [60]:
coupons = spark.sql('''SELECT
              CAST(_c0 AS BIGINT) as tkt_number,
              CAST(_c1 AS INT) as cpn_number,
              _c2 as origin,
              _c3 as dest,
              _c4 as carrier,
              CAST(_c6 AS FLOAT) as amount
              FROM csv. `/content/drive/My Drive/BigData Spark/Clase 1/coupon150720.csv.gz`''') 

#### From other types of data

Apache Parquet is a free and open-source column-oriented data store of the Apache Hadoop ecosystem. It is similar to the other columnar storage file formats available in Hadoop namely RCFile and Optimized RCFile. It is compatible with most of the data processing frameworks in the Hadoop environment.

In [59]:
spark.read.jdbc #(java database conexion) conectar un programa a una base de datos

<bound method DataFrameReader.jdbc of <pyspark.sql.readwriter.DataFrameReader object at 0x7ff4f961cdd8>>

### Basic operations with DataFrames

In [67]:
df.show(5) #muestra los n primeros

+---+------+
| _1|    _2|
+---+------+
|  0|hobbit|
|  1|   elf|
|  2|   elf|
|  3|   elf|
|  4|   orc|
+---+------+
only showing top 5 rows



In [68]:
df.take(3) #genera lista

[Row(_1=0, _2='hobbit'), Row(_1=1, _2='elf'), Row(_1=2, _2='elf')]

### Filtering and selecting

Syntax inspired in SQL.

In [75]:
df.select('id') #data frame con la columna id seria _1

DataFrame[id: bigint]

If we want to filter, we will need to build an instance of `Column`, using square bracket notation.

In [76]:
df['id'].show()

TypeError: ignored

In [77]:
df.select(df['id'])

DataFrame[id: bigint]

In [78]:
df.filter(df['id'] < 5)

DataFrame[id: bigint, race: string]

In [80]:
df['id'] < 5 #otra columna tipo booleano

Column<b'(id < 5)'>

That's because a comparison between str and int will error out, so spark will not even get the chance to infer to which column we are referring.

In [81]:
df.filter('id' < 5) #no se pueden comparar string con integer

TypeError: ignored

`where` is exactly synonimous with `filter`

In [ ]:
# df.where or df. filter es igual

In [82]:
df.where(df['id'] < 5)

DataFrame[id: bigint, race: string]

A column is quite different to a Pandas Series. It is just a reference to a column, and can only be used to construct sparkSQL expressions (select, where...). It can't be collected or taken as a one-dimensional sequence:

In [ ]:
df['race'].show() # no puedo pensar como en las series del dataframe de pandas

#### Exercise

Extract all employee ids which correspond to orcs

In [99]:
df.filter(df['race'] == 'orc').select('id').show() 

+---+
| id|
+---+
|  4|
|  5|
|  6|
| 18|
+---+



### Adding columns

Dataframes are immutable, since they are built on top of RDDs, so we can not assign to them. We need to create new DataFrames with the appropriate columns.

In [100]:
df2 = df.withColumn('square', df['id'] ** 2)
df2.show(5)

+---+------+------+
| id|  race|square|
+---+------+------+
|  0|hobbit|   0.0|
|  1|   elf|   1.0|
|  2|   elf|   4.0|
|  3|   elf|   9.0|
|  4|   orc|  16.0|
+---+------+------+
only showing top 5 rows



### User defined functions

There are many useful functions in pyspark.sql.functions. These work on columns, that is, they are vectorial.

We can write User Defined Functions (`udf`s), which allow us to "vectorize" operations: write a standard function to process single elements, then build a udf with that that works on columns in a DataFrame, like a SQL function.

In [101]:
df2['id'] + df2['square'] 

Column<b'(id + square)'>

In [102]:
import math

math.log1p(1)

0.6931471805599453

In [103]:
math.log1p(df['id']) #sabe manejar numeros no columnas

TypeError: ignored

This errors out because 

```python
math.log1p
```

is not a udf: it doesn't know how to work with strings or Column objects:

In [105]:
from pyspark.sql import functions #para operar con columnas 
 
df.select(functions.cos(df['id'])).show()

+--------------------+
|             COS(id)|
+--------------------+
|                 1.0|
|  0.5403023058681398|
| -0.4161468365471424|
| -0.9899924966004454|
| -0.6536436208636119|
| 0.28366218546322625|
|  0.9601702866503661|
|  0.7539022543433046|
|-0.14550003380861354|
| -0.9111302618846769|
| -0.8390715290764524|
|0.004425697988050785|
|  0.8438539587324921|
|  0.9074467814501962|
|  0.1367372182078336|
| -0.7596879128588213|
| -0.9576594803233847|
|-0.27516333805159693|
|  0.6603167082440802|
|  0.9887046181866692|
+--------------------+



But we can transform it into a udf:

In [107]:
my_udf = functions.udf(math.log1p) 
my_udf # es una funcion que come una funcion y devuelve una funcion

<function math.log1p>

In [109]:
df.select(my_udf(df['id'])).show()

+------------------+
|         log1p(id)|
+------------------+
|               0.0|
|0.6931471805599453|
|1.0986122886681096|
|1.3862943611198906|
|1.6094379124341003|
| 1.791759469228055|
|1.9459101490553132|
|2.0794415416798357|
|2.1972245773362196|
| 2.302585092994046|
|2.3978952727983707|
|2.4849066497880004|
|2.5649493574615367|
| 2.639057329615259|
|  2.70805020110221|
| 2.772588722239781|
| 2.833213344056216|
|2.8903717578961645|
|2.9444389791664403|
| 2.995732273553991|
+------------------+



We can do the same with any function we dream up:

In [110]:
last_2 = functions.udf(lambda word: word[-2:])
df.select(last_2('race')).show(5)

+--------------+
|<lambda>(race)|
+--------------+
|            it|
|            lf|
|            lf|
|            lf|
|            rc|
+--------------+
only showing top 5 rows



If we want the resulting columns to be of a particular type, we need to specify the return type. This is because in Python return types can not be inferred.

In [111]:
df.select(my_udf(df['id'])).printSchema() # Es tipo string 

root
 |-- log1p(id): string (nullable = true)



Think about this function: what is its return type?

#### Exercise: 

Create a 'hitpoints' field in our df. make it 30000 for halflings, 40000 for elves and 70000 for orcs.



In [113]:
 from pyspark.sql import functions 

def hitpoints(race):
    reference = {'elf': 40000, 'orc' : 70000, 'hobbit' : 30000}
    return reference[race]

hitpoints('hobbit')

30000

In [117]:
hp_udf = functions.udf(hitpoints, types.IntegerType())
hp_udf(df['race'])

Column<b'hitpoints(race)'>

In [119]:
df3 = df.withColumn('hp', hp_udf('race')) #añado la columna hp, aplico la variable hp_udf que tiene una función
df3.show()

+---+------+-----+
| id|  race|   hp|
+---+------+-----+
|  0|hobbit|30000|
|  1|   elf|40000|
|  2|   elf|40000|
|  3|   elf|40000|
|  4|   orc|70000|
|  5|   orc|70000|
|  6|   orc|70000|
|  7|   elf|40000|
|  8|hobbit|30000|
|  9|   elf|40000|
| 10|   elf|40000|
| 11|hobbit|30000|
| 12|   elf|40000|
| 13|   elf|40000|
| 14|hobbit|30000|
| 15|hobbit|30000|
| 16|   elf|40000|
| 17|hobbit|30000|
| 18|   orc|70000|
| 19|   elf|40000|
+---+------+-----+



If we have a column that is not the desired type, we can convert it with `cast`.

### Summary statistics

https://databricks.com/blog/2015/06/02/statistical-and-mathematical-functions-with-dataframes-in-spark.html

In [120]:
df3.stat.cov('id', 'hp')

-12105.263157894737

### .crosstab()

Crosstab returns the contingency table for two columns, as a DataFrame.

In [123]:
land = functions.udf(lambda: random.choice(['gondor', 'rohan']) ) # a partir de nada creo aleatoriamente

df4 = df3.withColumn('land', land())
df4.show()
df4.cache().show() #para que no cambie el número aleaotorio

+---+------+-----+------+
| id|  race|   hp|  land|
+---+------+-----+------+
|  0|hobbit|30000| rohan|
|  1|   elf|40000| rohan|
|  2|   elf|40000| rohan|
|  3|   elf|40000| rohan|
|  4|   orc|70000|gondor|
|  5|   orc|70000|gondor|
|  6|   orc|70000| rohan|
|  7|   elf|40000|gondor|
|  8|hobbit|30000|gondor|
|  9|   elf|40000|gondor|
| 10|   elf|40000|gondor|
| 11|hobbit|30000| rohan|
| 12|   elf|40000| rohan|
| 13|   elf|40000| rohan|
| 14|hobbit|30000| rohan|
| 15|hobbit|30000|gondor|
| 16|   elf|40000| rohan|
| 17|hobbit|30000|gondor|
| 18|   orc|70000|gondor|
| 19|   elf|40000| rohan|
+---+------+-----+------+



In [127]:
df4.crosstab('race', 'land').show()

+---------+------+-----+
|race_land|gondor|rohan|
+---------+------+-----+
|   hobbit|     4|    2|
|      orc|     3|    1|
|      elf|     3|    7|
+---------+------+-----+



### Grouping

Grouping works very similarly to Pandas: executing groupby (or groupBy) on a DataFrame will return an object (a GroupedData) that can then be aggregated to obtain the results.

In [129]:
gd = df4.groupBy('land')
gd

GroupedData has several aggregation functions defined:

In [130]:
gd.sum('hp').show()

+------+-------+
|  land|sum(hp)|
+------+-------+
| rohan| 520000|
|gondor| 340000|
+------+-------+



We can do several aggregations in a single step, with a number of different syntaxes:

In [131]:
gd.agg({'hp' : 'mean', 'id' : 'count'}).show()

+------+-------+---------+
|  land|avg(hp)|count(id)|
+------+-------+---------+
| rohan|45000.0|       10|
|gondor|41000.0|       10|
+------+-------+---------+



In [125]:
gd.agg(functions.mean('hp'),
       functions.count('id'),
       functions.mean('id')).show()

+------+-------+---------+-------+
|  land|avg(hp)|count(id)|avg(id)|
+------+-------+---------+-------+
| rohan|41000.0|       10|    7.4|
|gondor|45000.0|       10|   11.6|
+------+-------+---------+-------+



In [132]:
df4.groupby(df4['id'] < 5).mean('hp').show()

+--------+------------------+
|(id < 5)|           avg(hp)|
+--------+------------------+
|    true|           44000.0|
|   false|42666.666666666664|
+--------+------------------+



### Intersections

Ver much like SQL joins. We can specify the columns and the join method (left, right, inner, outer) or we can let Spark infer them.

In [134]:
result = gd.agg(functions.mean('hp'),
       functions.count('id'),
       functions.mean('id'))
result.show()



+------+------------------+---------+-----------------+
|  land|           avg(hp)|count(id)|          avg(id)|
+------+------------------+---------+-----------------+
| rohan|           45000.0|        8|            10.75|
|gondor|41666.666666666664|       12|8.666666666666666|
+------+------------------+---------+-----------------+



In [135]:
df4.join(result).show()

AnalysisException: ignored

Spark refuses to do cross joins by default. To perform them, we can 

a) Allow then explicitly:

```python
spark.conf.set("spark.sql.crossJoin.enabled", "true")
```

b) Specify the join criterion

```python
df4.join(new_df, on='id').show()
```

In [136]:
df4.join(result, on='land').show()

+------+---+------+-----+-------+---------+-------+
|  land| id|  race|   hp|avg(hp)|count(id)|avg(id)|
+------+---+------+-----+-------+---------+-------+
| rohan|  0|hobbit|30000|41000.0|       10|    7.4|
| rohan|  1|   elf|40000|41000.0|       10|    7.4|
| rohan|  2|   elf|40000|41000.0|       10|    7.4|
| rohan|  3|   elf|40000|41000.0|       10|    7.4|
| rohan|  6|   orc|70000|41000.0|       10|    7.4|
| rohan| 10|   elf|40000|41000.0|       10|    7.4|
| rohan| 12|   elf|40000|41000.0|       10|    7.4|
| rohan| 14|hobbit|30000|41000.0|       10|    7.4|
| rohan| 15|hobbit|30000|41000.0|       10|    7.4|
| rohan| 19|   elf|40000|41000.0|       10|    7.4|
|gondor|  4|   orc|70000|45000.0|       10|   11.6|
|gondor|  5|   orc|70000|45000.0|       10|   11.6|
|gondor|  7|   elf|40000|45000.0|       10|   11.6|
|gondor|  8|hobbit|30000|45000.0|       10|   11.6|
|gondor|  9|   elf|40000|45000.0|       10|   11.6|
|gondor| 11|hobbit|30000|45000.0|       10|   11.6|
|gondor| 13|

In [137]:
df4.join(result, on=df4['id'] > result['count(id)']).show()

+---+------+-----+------+------+------------------+---------+-------+
| id|  race|   hp|  land|  land|           avg(hp)|count(id)|avg(id)|
+---+------+-----+------+------+------------------+---------+-------+
|  9|   elf|40000| rohan| rohan|           37500.0|        8|  9.875|
| 10|   elf|40000| rohan| rohan|           37500.0|        8|  9.875|
| 11|hobbit|30000| rohan| rohan|           37500.0|        8|  9.875|
| 12|   elf|40000| rohan| rohan|           37500.0|        8|  9.875|
| 13|   elf|40000|gondor| rohan|           37500.0|        8|  9.875|
| 14|hobbit|30000| rohan| rohan|           37500.0|        8|  9.875|
| 15|hobbit|30000|gondor| rohan|           37500.0|        8|  9.875|
| 16|   elf|40000|gondor| rohan|           37500.0|        8|  9.875|
| 17|hobbit|30000|gondor| rohan|           37500.0|        8|  9.875|
| 18|   orc|70000| rohan| rohan|           37500.0|        8|  9.875|
| 19|   elf|40000|gondor| rohan|           37500.0|        8|  9.875|
| 13|   elf|40000|go

In [138]:
df4.join(result, on=df4['id'] > result['count(id)'], how='left').show()

+---+------+-----+------+------+-------+---------+-------+
| id|  race|   hp|  land|  land|avg(hp)|count(id)|avg(id)|
+---+------+-----+------+------+-------+---------+-------+
|  0|hobbit|30000| rohan|  null|   null|     null|   null|
|  1|   elf|40000| rohan|  null|   null|     null|   null|
|  2|   elf|40000|gondor|  null|   null|     null|   null|
|  3|   elf|40000|gondor|  null|   null|     null|   null|
|  4|   orc|70000|gondor|  null|   null|     null|   null|
|  5|   orc|70000|gondor|  null|   null|     null|   null|
|  6|   orc|70000| rohan|  null|   null|     null|   null|
|  7|   elf|40000|gondor|  null|   null|     null|   null|
|  8|hobbit|30000|gondor|  null|   null|     null|   null|
|  9|   elf|40000|gondor|  null|   null|     null|   null|
| 10|   elf|40000|gondor|  null|   null|     null|   null|
| 11|hobbit|30000|gondor| rohan|36000.0|       10|   11.5|
| 11|hobbit|30000|gondor|gondor|50000.0|       10|    7.5|
| 12|   elf|40000| rohan| rohan|36000.0|       10|   11.

#### Digression: Don't work in GoogleColaboratory

We can monitor our running jobs and storage used at the Spark Web UI. We can get its url with sc.uiWebUrl.

StorageLevels represent how our DataFrame is cached: we can save the results of the computation up to that point, so that if we process several times the same data only the subsequent steps will be recomputed.

We can erase it with `unpersist`

#### Exercise

Calculate the [z-score](http://www.statisticshowto.com/probability-and-statistics/z-score/) of each employee's hitpoints for their location

1) Calculate the mean and std of hitpoints for each location

2) Annotate each employee with the stats corresponding to their location

3) Calculate the z-score

Note that we can build more complex boolean conditions for joining, as well as joining on columns that do not have the same name:

### Handling null values

## SQL querying

We need to register our DataFrame as a table in the SQL context in order to be able to query against it.

Once registered, we can perform queries as complex as we want.

## Interoperation with Pandas

Easy peasy. We can convert a spark DataFrame into a Pandas one, which will `collect` it, and viceversa, which will distribute it.

## Writing out


#### Exercise

Repeat the exercise from the previous notebook, but this time with DataFrames.

Get stats for all tickets with destination MAD from `coupons150720.csv`.

You will need to extract ticket amounts with destination MAD, and then calculate:

1. Total ticket amounts per origin
2. Top 10 airlines by average amount

1) Extract the fields you need (c0,c1,c2,c3,c4 and c6) into a dataframe with proper names and types

Remember, you want to calculate:

Total ticket amounts per origin

Top 10 airlines by average amount

2) Total ticket amounts per origin

3) Top 10 Airlines by average amount



## Further Reading

https://databricks.com/blog/2016/07/14/a-tale-of-three-apache-spark-apis-rdds-dataframes-and-datasets.html

https://www.datacamp.com/community/tutorials/apache-spark-python

https://spark.apache.org/docs/2.2.0/sql-programming-guide.html

https://ogirardot.wordpress.com/2015/05/29/rdds-are-the-new-bytecode-of-apache-spark/

https://stackoverflow.com/questions/36822224/what-are-the-pros-and-cons-of-parquet-format-compared-to-other-formats

https://s3.amazonaws.com/assets.datacamp.com/blog_assets/PySpark_SQL_Cheat_Sheet_Python.pdf